# Validation of method

### Table of contents

1. [**Schematic of models**](#schem)
2. [**Synthetic**](#syn)
3. [**Biophysical**](#bio)
4. [**Place cells regression**](#place)
5. [**Place cells comodulation**](#corr)

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy

import equinox as eqx
import jax
import jax.numpy as jnp
import jax.random as jr

import optax
jax.config.update('jax_platform_name', 'cpu')
#jax.config.update("jax_enable_x64", True)

import pickle


import sys

sys.path.append("../../../GaussNeuro")
import gaussneuro as lib

sys.path.append("../../scripts/fit")

import template



import matplotlib.pyplot as plt
plt.style.use(['paper.mplstyle'])
#plt.rc('text', usetex=True)

/var/home/dl543/Documents/Learning/nonparametric-nonrenewal-process/scripts/plots/../../../GaussNeuro/gaussneuro/likelihoods/renewal.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
jax.config.update('jax_platform_name', 'cpu')
#jax.config.update('jax_disable_jit', True)

double_arrays = False

if double_arrays:
    jax.config.update("jax_enable_x64", True)
    array_type = "float64"
else:
    array_type = "float32"

# seed everything
seed = 123
prng_state = jr.PRNGKey(seed)
rng = np.random.default_rng(seed)

In [4]:
import plot_real

## hc3

In [5]:
datadir = '../saves/'

tuning_dict = pickle.load(
    open(datadir + 'hc3_tuning' + ".p", "rb")
)

variability_dict = pickle.load(
    open(datadir + 'hc3_variability' + ".p", "rb")
)

bnpp_dict = pickle.load(
    open(datadir + 'hc3_regression' + ".p", "rb")
)

baseline_dict = pickle.load(
    open(datadir + 'hc3_baselines' + ".p", "rb")
)

regression_dict = {**baseline_dict, **bnpp_dict}
reg_config_names = list(regression_dict.keys())

    
plot_units = [1, 15]

In [7]:
cs = [
    'tab:blue',
    'tab:orange', 
    'tab:green',
    'tab:red',
    'tab:purple',
    'tab:brown',
    'tab:pink',
    'tab:gray',
]
use_model_inds = np.array([0, 1, 2, 3, 4, 5, 6, 14])
use_reg_config_names = [reg_config_names[k] for k in use_model_inds]
use_names = [
    'Poisson', 
    'gamma', 
    'inv. Gauss.', 
    'RC cond. P', 
    'RC cond. G', 
    'RC cond. IG', 
    'NP cond. P', 
    'NP (ours)', 
]
visualize_inds = [0, 1, 4, 6, 7]
visualize_names = [reg_config_names[k] for k in use_model_inds[visualize_inds]]


In [9]:
use_reg_config_names

['ec014.29_ec014.468_isi5ISI5sel0.0to0.5_PP-log__factorized_gp-32-1000_X[x-hd-theta]_Z[]_freeze[]',
 'ec014.29_ec014.468_isi5ISI5sel0.0to0.5_gamma-log__rate_renewal_gp-32-1000_X[x-hd-theta]_Z[]_freeze[]',
 'ec014.29_ec014.468_isi5ISI5sel0.0to0.5_invgauss-log__rate_renewal_gp-32-1000_X[x-hd-theta]_Z[]_freeze[]',
 'ec014.29_ec014.468_isi5ISI5sel0.0to0.5_PP-log_rcb-8-10.-20.-4.5-9.-self-H150_factorized_gp-32-1000_X[x-hd-theta]_Z[]_freeze[obs_model0spikefilter0a-obs_model0spikefilter0log_c-obs_model0spikefilter0phi]',
 'ec014.29_ec014.468_isi5ISI5sel0.0to0.5_gamma-log_rcb-8-10.-20.-4.5-9.-self-H150_rate_renewal_gp-32-1000_X[x-hd-theta]_Z[]_freeze[obs_model0spikefilter0a-obs_model0spikefilter0log_c-obs_model0spikefilter0phi]',
 'ec014.29_ec014.468_isi5ISI5sel0.0to0.5_invgauss-log_rcb-8-10.-20.-4.5-9.-self-H150_rate_renewal_gp-32-1000_X[x-hd-theta]_Z[]_freeze[obs_model0spikefilter0a-obs_model0spikefilter0log_c-obs_model0spikefilter0phi]',
 'ec014.29_ec014.468_isi5ISI5sel0.0to0.5_PP-log_svgp-